<a href="https://colab.research.google.com/github/shubhamsks/deep-learning/blob/master/cnn_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In  this cell we will make Convulation Layer using tensor flow to take the image of dimesnion as specified  as input

In [0]:
import tensorflow as tf
# depth of the output
k_output = 64

# image dimensions 
image_width = 10
image_height = 10
color_channels = 3
# convolution filter dimension
filter_size_width = 5
filter_size_height = 5
# input /image
input = tf.placeholder(tf.float32, shape = [None, image_height, image_width, color_channels])

In [0]:
# weight and bias
weight = tf.Variable(tf.truncated_normal([filter_size_height, filter_size_width,color_channels, k_output]))

bias = tf.Variable(tf.zeros(k_output))

In [0]:
# apply convolution 
conv_layer = tf.nn.conv2d(input, weight,strides = [1,2,2,1], padding = 'SAME')
# add bias
conv_layer = tf.nn.bias_add(conv_layer, bias)
conv_layer = tf.nn.relu(conv_layer)


TensorFlow uses a ```stride``` for each input dimension, 

```[batch, input_height, input_width, input_channels]```


We generally always set the stride for batch and input_channels (i.e. the first and fourth element in the strides array) to be 1. This ensures that the model uses all batches and input channels. (It's good practice to remove the batches or channels you want to skip from the data set rather than use a stride to skip them.)

In [0]:
"""
Setup the strides, padding and filter weight/bias such that
the output shape is (1, 2, 2, 3).
"""
import tensorflow as tf
import numpy as np

# `tf.nn.conv2d` requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)


def conv2d(input):
    # Filter (weights and bias)
    # The shape of the filter weight is (height, width, input_depth, output_depth)
    # The shape of the filter bias is (output_depth,)
    # TODO: Define the filter weights `F_W` and filter bias `F_b`.
    # NOTE: Remember to wrap them in `tf.Variable`, they are trainable parameters after all.
    F_W = tf.Variable(tf.truncated_normal([2, 2, 1, 3]))
    F_b = tf.zeros(3)
    # TODO: Set the stride for each dimension (batch_size, height, width, depth)
    strides = [1, 2, 2, 1]
    # TODO: set the padding, either 'VALID' or 'SAME'.
    padding = 'SAME'
    # https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#conv2d
    # `tf.nn.conv2d` does not include the bias computation so we have to add it ourselves after.
    return tf.nn.conv2d(input, F_W, strides, padding) + F_b

out = conv2d(X)

out


<tf.Tensor 'add:0' shape=(1, 2, 2, 3) dtype=float32>

In [0]:
def conv2d(input):
    # Filter (weights and bias)
    # The shape of the filter weight is (height, width, input_depth, output_depth)
    # The shape of the filter bias is (output_depth,)
    # NOTE: Remember to wrap them in `tf.Variable`, they are trainable parameters after all.
    F_W = tf.Variable(tf.truncated_normal([2,2,1,3]) )
    F_b = tf.Variable(tf.zeros(3))
    strides = [1,2,2,1]
    padding = 'VALID'
    # https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#conv2d
    # `tf.nn.conv2d` does not include the bias computation so we have to add it ourselves after.
    return tf.nn.conv2d(input, F_W, strides, padding) + F_b

weights = tf.Variable(tf.truncated_normal([2,2,1,3]))
bias = tf.Variable(tf.zeros(3))
strides = [1,2,2,1]
padding ='SAME'
conv_layer = tf.nn.conv2d(X, weights,strides,padding) + bias
out = conv2d(X)

In [0]:
conv_layer

<tf.Tensor 'add_1:0' shape=(1, 2, 2, 3) dtype=float32>

In [0]:
conv_layer = tf.nn.relu(conv_layer)
conv_layer = tf.nn.max_pool(conv_layer,
                           ksize = [1,2,2,1],
                           strides = [1,2,2,1],
                           padding = 'SAME')
# ksize parameters corresponds to [batch, height, width, channels]


In [56]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".",one_hot = True, reshape = False)

import tensorflow as tf

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [0]:
# parameters 
learning_rate = 0.00001
epochs = 10
batch_size = 128
# no. of samples to calculate validation and accuracy
test_valid_size = 256

# network params
n_classes = 10
dropout = 0.75
weights = {
    'wc1':tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2':tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1':tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out':tf.Variable(tf.random_normal([1024, n_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

In [0]:
def conv2d(x, w, b, strides = 1):
  x = tf.nn.conv2d(x, w,strides = [1, strides, strides, 1],padding = 'SAME')
  x = tf.nn.bias_add(x, b)
  return tf.nn.relu(x)

In [0]:
def maxpool2d(x,k = 2):
  return tf.nn.max_pool(x,
                    ksize = [1,k,k,1],
                    strides=[1,k,k,1],
                    padding='SAME')


In [0]:

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [55]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

W0819 05:35:05.973325 140436430251904 deprecation.py:323] From <ipython-input-55-86dc9ed7e0ea>:9: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Epoch  1, Batch   1 -Loss: 76112.3906 Validation Accuracy: 0.117188
Epoch  1, Batch   2 -Loss: 66482.7188 Validation Accuracy: 0.125000
Epoch  1, Batch   3 -Loss: 54126.0703 Validation Accuracy: 0.171875
Epoch  1, Batch   4 -Loss: 43945.2109 Validation Accuracy: 0.164062
Epoch  1, Batch   5 -Loss: 34409.0781 Validation Accuracy: 0.183594
Epoch  1, Batch   6 -Loss: 38636.6680 Validation Accuracy: 0.175781
Epoch  1, Batch   7 -Loss: 35253.4453 Validation Accuracy: 0.171875
Epoch  1, Batch   8 -Loss: 31167.4941 Validation Accuracy: 0.171875
Epoch  1, Batch   9 -Loss: 31093.7344 Validation Accuracy: 0.167969
Epoch  1, Batch  10 -Loss: 29439.9961 Validation Accuracy: 0.183594
Epoch  1, Batch  11 -Loss: 29053.7070 Validation Accuracy: 0.195312
Epoch  1, Batch  12 -Loss: 27574.4121 Validation Accuracy: 0.171875
Epoch  1, Batch  13 -Loss: 26135.3594 Validation Accuracy: 0.179688
Epoch  1, Batch  14 -Loss: 27706.1309 Validation Accuracy: 0.183594
Epoch  1, Batch  15 -Loss: 25866.3438 Validation